Import libraries

In [49]:
import numpy as np
import pandas as pd
import os
import pickle
import glob
from collections import defaultdict
import matplotlib.pyplot as plt
from hoi.metrics import InfoTopo, TC, DTC, Sinfo, Oinfo, RedundancyphiID, SynergyphiID
from hoi.core import entropy_hist

Define functions

In [122]:
# --- Core Functions ---
def compute_information_metric(metric_class, data):
    """Compute information metric on data using a given metric class."""
    data_array = np.array(data, dtype=float)
    model = metric_class(data_array)
    result = model.fit(minsize=1, maxsize=5)
    return result, model

def divide_list_by_order(values, model):
    """Divide values into groups by interaction order."""
    orders = model.order
    grouped = defaultdict(list)
    for order, val in zip(orders, values):
        grouped[order].append(val.item())
    return {k: sorted(v) for k, v in grouped.items()}

def analyse_information_metric(metric_class, data):
    """Analyze data using a specified metric and return grouped results."""
    values, model = compute_information_metric(metric_class, data)
    return divide_list_by_order(values, model)

def sliding_window_analysis(metric_class, timeseries, window_size, step_size):
    """Apply a metric using a sliding window over a time series."""
    results = []
    for start in range(0, len(timeseries) - window_size, step_size):
        window_data = timeseries[start:start + window_size]
        result = analyse_information_metric(metric_class, window_data)
        results.append(dict(result))
    return results

def average_metric_data(simulations):
    """Prepare averaged metric data for heatmap visualization."""
    key_indices = set(
        (k, idx) for sim in simulations for step in sim for k, v in step.items() for idx in range(len(v))
    )
    sorted_keys = sorted(key_indices)

    data = []
    for t in range(len(simulations[0])):
        avg_step = {
            (k, idx): np.mean([sim[t].get(k, [0.0]*(idx+1))[idx] for sim in simulations])
            for k, idx in sorted_keys
        }
        data.append([avg_step[(k, idx)] for k, idx in sorted_keys])

    label_pos = {k: max(i for i, (key, _) in enumerate(sorted_keys) if key == k) for k, _ in sorted_keys}
    labels = list(label_pos.keys())

    return np.array(data), labels, label_pos

def compute_simulation_averages(metric_sims):
    """Average metric values across simulations and time."""
    all_sim_averages = [
        [np.mean([val for values in step.values() for val in values]) for step in sim]
        for sim in metric_sims
    ]
    combined_avg = np.mean(all_sim_averages, axis=0)
    return (*all_sim_averages, combined_avg)

# --- Additional Metric Functions ---
def compute_synergy_redundancy_CR(metric_results):
    redundancy_new = []
    synergy_new = []

    for sim1, sim2 in zip(metric_results['InfoTopo'], metric_results['TC']):
        redundancy_sim = []
        synergy_sim = []
        for step1, step2 in zip(sim1, sim2):
            keys = set(step1.keys()) | set(step2.keys())
            redundancy_step = {}
            synergy_step = {}
            for k in keys:
                vals1 = step1.get(k, [])
                vals2 = step2.get(k, [])
                max_len = max(len(vals1), len(vals2))
                v1 = np.array(vals1 + [0.0] * (max_len - len(vals1)))
                v2 = np.array(vals2 + [0.0] * (max_len - len(vals2)))
                redundancy_step[k] = ((v1 + v2) / 2).tolist()
                synergy_step[k] = ((v2 - v1) / 2).tolist()
            redundancy_sim.append(redundancy_step)
            synergy_sim.append(synergy_step)
        redundancy_new.append(redundancy_sim)
        synergy_new.append(synergy_sim)

    metric_results["Redundancy_CR"] = redundancy_new
    metric_results["Synergy_CR"] = synergy_new

def compute_average_TC(metric_results):
    TC_average = []
    for sim in metric_results['TC']:
        TC_sim = []
        for step in sim:
            TC_step = {}
            for k, vals in step.items():
                if vals:  # Avoid division by zero
                    avg = sum(vals) / len(vals)
                    TC_step[k] = [avg] * len(vals)
                else:
                    TC_step[k] = []
            TC_sim.append(TC_step)
        TC_average.append(TC_sim)
    metric_results["TC_average"] = TC_average

def compute_tse_values(TC_dict):
    """
    Computes TSE complexity values for each subset of the input TC_dict.
    For each subset size N, this function computes TSE values assuming
    only one value is used for the total correlation of size N at a time.

    Parameters:
        TC_dict (dict): Dictionary where keys are integers representing subset sizes,
                        and values are lists of total correlations.

    Returns:
        dict: TSE_values where each key is a subset size, and the value is a list
              of TSE complexities corresponding to each value in TC_dict[N].
    """
    
    def compute_tse_complexity(sub_TC_dict):
        N = max(sub_TC_dict.keys())  # Maximum subset size
        TSE = 0

        TC_full_list = sub_TC_dict.get(N, [])
        if not TC_full_list:
            raise ValueError(f"No total correlation data found for full set size N={N}.")
        TC_full = np.mean(TC_full_list)

        for gamma in range(1, N):
            subset_tcs = sub_TC_dict.get(gamma, [])
            if not subset_tcs:
                continue
            E_TC_gamma = np.mean(subset_tcs)
            TSE += (gamma / N) * TC_full - E_TC_gamma

        return TSE

    TSE_values = {}

    for max_k in sorted(TC_dict.keys()):
        partial_dict = {k: TC_dict[k] for k in range(1, max_k + 1)}
        TSE_values[max_k] = []
        for i in range(len(TC_dict[max_k])):
            temp_dict = {k: TC_dict[k] for k in range(1, max_k)}
            temp_dict[max_k] = [TC_dict[max_k][i]]
            try:
                tse = compute_tse_complexity(temp_dict)
                TSE_values[max_k].append(tse)
            except ValueError as e:
                TSE_values[max_k].append(None)
                print(f"Error for key {max_k}, index {i}: {e}")

    return TSE_values

def compute_TSE_complexity(metric_results):
    TSE_values = []
    for sim in metric_results['TC']:
        TSE_sim = []
        for step in sim:
            print(step)
            TSE_step = compute_tse_values(step)
            TSE_sim.append(TSE_step)
        TSE_values.append(TSE_sim)
    metric_results["TSE_complexity"] = TSE_values

# --- Data Loading ---
def load_csv_files_from_directory(directory):
    return glob.glob(os.path.join(directory, '*.csv'))

# --- Main Execution ---
def main(data_dir, metrics_classes=[InfoTopo, TC, Oinfo], timeseries_percentage=1, window_percentage=0.5, step_percentage=0.1):
    files = load_csv_files_from_directory(data_dir)

    metric_results = {}
    for metric in metrics_classes:
        metric_name = metric.__name__
        all_sims = []
        for file_path in files:
            df = pd.read_csv(file_path)
            df_length = int(timeseries_percentage * len(df))
            df = df[:df_length]
            window = int(window_percentage * len(df))
            step = int(step_percentage * len(df))
            sim_result = sliding_window_analysis(metric, df, window, step)
            all_sims.append(sim_result)
        metric_results[metric_name] = all_sims

    compute_synergy_redundancy_CR(metric_results)
    compute_average_TC(metric_results)
    compute_TSE_complexity(metric_results)

    heatmap_data = {k: average_metric_data(v) for k, v in metric_results.items()}
    averages_data = {k: compute_simulation_averages(v) for k, v in metric_results.items()}

    heatmap_path = os.path.join(data_dir, "heatmap_data.pkl")
    averages_path = os.path.join(data_dir, "averages_data.pkl")

    with open(heatmap_path, "wb") as f:
        pickle.dump(heatmap_data, f)

    with open(averages_path, "wb") as f:
        pickle.dump(averages_data, f)

Check the length of the timeseries in the desired dataset

In [118]:
directory = r'Data\Kuramoto_data\Traditional'
csv_files = glob.glob(os.path.join(directory, '*.csv'))
print(csv_files[0])
df = pd.read_csv(csv_files[0])
len(df)

Data\Kuramoto_data\Traditional\kuramoto_frequencies_timeseries_N8_K3.0_T20_1.csv


2000

Compute and store the desired metrics

In [125]:
data_directory = r'Data\Kuramoto_data\Traditional'
metrics = [InfoTopo, TC, DTC, Sinfo, Oinfo, RedundancyphiID, SynergyphiID]
timeseries_percentage = 1
window_percentage = 0.5
step_percentage = 0.05

main(data_directory, metrics, timeseries_percentage, window_percentage, step_percentage)

Compute entropy with gc
    Copnorm and demean the data
Get list of multiplets


  0%|          |  0/5 [00:00<?,       ?it/s]

Get list of multiplets


  0%|          | 0/218 [00:00<?, ?it/s]

Compute entropy with gc
    Copnorm and demean the data
Get list of multiplets


...

    Copnorm and demean the data
Get list of multiplets


  0%|          |  0/4 [00:00<?,       ?it/s]

    Copnorm and demean the data
Get list of multiplets


  0%|          |  0/4 [00:00<?,       ?it/s]

{1: [-5.8650755363487406e-08, -5.8650755363487406e-08, -5.8650755363487406e-08, -5.8650755363487406e-08, -5.8650755363487406e-08, -3.1325612326327246e-08, -3.1325612326327246e-08, -3.1325612326327246e-08], 2: [0.00023512382176704705, 0.002780467038974166, 0.016614539548754692, 0.045535143464803696, 0.04685768857598305, 0.049970708787441254, 0.057310108095407486, 0.06041886657476425, 0.0736629068851471, 0.18267492949962616, 0.2144055962562561, 0.21457496285438538, 0.22033658623695374, 0.23263141512870789, 0.2739650309085846, 0.29568347334861755, 0.35659149289131165, 0.4745294749736786, 0.4971407353878021, 0.5535780787467957, 0.5602537393569946, 0.5979733467102051, 0.6507420539855957, 1.0553947687149048, 1.7883399724960327, 2.1820459365844727, 2.538770914077759, 3.0670950412750244], 3: [0.28928688168525696, 0.3118998408317566, 0.32444149255752563, 0.3534061312675476, 0.40579918026924133, 0.44673895835876465, 0.4494370222091675, 0.49586614966392517, 0.5110907554626465, 0.5821368098258972,